In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_validate

red_path = "./res/winequality-red.csv"
white_path = "./res/winequality-white.csv"
result_folder = "./results/"

test_set_ratio = 0.25 
random_state = 42

In [15]:
## preprocessing
## read red and white wine
df_red = pd.read_csv(red_path,";")
df_white = pd.read_csv(white_path, ";")


#df_ = pd.concat([df_red])
#df_ = pd.concat([df_white])
df_ = pd.concat([df_red,df_white])


## add ID column
df_['ID'] = df_.index


## create train/test for both for prediction (not training it)
df_train, df_test = train_test_split(df_, shuffle=True, test_size=test_set_ratio, random_state=random_state)
df_train = df_train.sort_values(by=['ID']).reset_index(drop=True)
df_test = df_test.sort_values(by=['ID']).reset_index(drop=True)

In [ ]:
#################### Separate red/white train test sets
## add ID column
#df_red['ID'] = df_red.index
#df_white['ID'] = df_red.shape[0] + df_white.index

## create train/test for both for testing the classifier (not training it)
#df_red_train, df_red_test = train_test_split(df_red, shuffle=True, test_size=test_set_ratio, random_state=random_state)
#df_red_train = df_red_train.sort_values(by=['ID']).reset_index(drop=True)
#df_red_test = df_red_test.sort_values(by=['ID']).reset_index(drop=True)

#df_white_train, df_white_test = train_test_split(df_white, shuffle=True, test_size=test_set_ratio, random_state=random_state)
#df_white_train = df_white_train.sort_values(by=['ID']).reset_index(drop=True)
#df_white_test = df_white_test.sort_values(by=['ID']).reset_index(drop=True)

In [16]:
df_train.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,ID
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
3,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,2
4,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
5,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,3
6,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
7,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4
8,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,4
9,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,5


### Initialize Training data

In [17]:
df_vars = df_train.loc[:, df_train.columns != 'quality']
df_class = df_train['quality']

X_train, X_test, y_train, y_test = train_test_split(df_vars, df_class, test_size=test_set_ratio, random_state=0)
clf = SVC(gamma='auto')

### Simple cross validation for well-founded results

In [18]:
scores = cross_validate(clf, df_vars, df_class, cv=5)
scores['test_score']

D:\Program_Files\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


array([0.42827869, 0.42666667, 0.43076923, 0.42607803, 0.43106996])

### Model fitting and sample testing

In [19]:
model = clf.fit(X_train, y_train)
model.score(X_test, y_test)

0.49507389162561577

### Final prediction

In [20]:
df_res = pd.DataFrame()
df_res['ID'] = df_test['ID']
df_res['quality'] = pd.Series(model.predict(df_test.loc[:,df_test.columns != 'quality']))
df_res.head(10)
np.count_nonzero(df_res['quality']==df_test['quality'])

864

### Export

In [ ]:
export_file_name = result_folder + "simple_svm.csv"
df_res.to_csv(export_file_name, index=False)